In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import shuffle
# >> FEATURE SELECTION << #


# >> MODEL TRAINING << #
def compute_cost(W, X, Y):
    # calculate hinge loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    print("distances")
    
    
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    print(distances[0:10])
    print(len(distances))
    hinge_loss = regularization_strength * (np.sum(distances) / N)
    #print(hinge_loss)

    # calculate cost
    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost


# I haven't tested it but this same function should work for
# vanilla and mini-batch gradient descent as well
def calculate_cost_gradient(W, X_batch, Y_batch):

    distance = 1 - (Y_batch * np.dot(X_batch, W))


    # dw = np.zeros(len(W))

    if distance < 0:
        di = W
    else:
        di = W - (regularization_strength * Y_batch * X_batch)
       
  
    return di


def sgd(features, outputs):
    max_epochs = 5000
    print("sgd")
    weights = np.zeros(features.shape[1])
    #print(weights)
    nth = 0
    prev_cost = float("inf")
    print(prev_cost)
    cost_threshold = 0.01  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(features, outputs)
        for ind, x in enumerate(X):
           
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            
          
            weights = weights - (learning_rate * ascent)
        # print(ascent)
        # print("ascent")
        # print(weights)
            

        # convergence check on 2^nth epoch
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            cost = compute_cost(weights, features, outputs)
            print("Epoch is: {} and Cost is: {}".format(epoch, cost))
            # stoppage criterion
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights
            prev_cost = cost
            nth += 1
    return weights


########################


def init():

    print("reading dataset...")
    # read data in pandas (pd) data frame
    data = pd.read_table('mixed200.txt')
    data = data.drop("No.", axis=1)
    data = data.drop("StdPageRank", axis=1)
    data = data.drop("VarPageRank", axis=1)


    state_map = {1: 1.0, 0: -1.0}
    data['State'] = data['State'].map(state_map)

    print(data['State'])

    # put features & outputs in different data frames

    training = 57748
    testing = 24749
    Y = data.loc[:, 'State']
    X = data.iloc[:, 0:9]
  
    
    # filter features
    # remove_correlated_features(X)
    # remove_less_significant_features(X, Y)

    # normalize data for better convergence and to prevent overflow
    X_normalized = MinMaxScaler().fit_transform(X.values)
    X = pd.DataFrame(X_normalized)

    # insert 1 in every row for intercept b
    X.insert(loc=len(X.columns), column='intercept', value=1)

    # split data into train and test set
    print("splitting dataset into train and test sets...")
    X_train, X_test, y_train, y_test = X[0:training],X[training:testing+training],Y[0:training],Y[training:testing+training]
    

    # train the model
    print("training started...")
    print(X_train.to_numpy())
    print(y_train.to_numpy())
    W = sgd(X_train.to_numpy(), y_train.to_numpy())
    print("training finished.")
    print("weights are: {}".format(W))

    # testing the model
    print("testing the model...")


    y_test_predicted = np.array([])
    for i in range(X_test.shape[0]):
        yp = np.sign(np.dot(X_test.to_numpy()[i], W))
        y_test_predicted = np.append(y_test_predicted, yp)

    print("accuracy on test dataset: {}".format(accuracy_score(y_test, y_test_predicted)))
    print("recall on test dataset: {}".format(recall_score(y_test, y_test_predicted)))
    print("precision on test dataset: {}".format(recall_score(y_test, y_test_predicted)))


# set hyper-parameters and call init
regularization_strength = 10000
learning_rate = 0.000001
init()


reading dataset...
0       -1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
82492    1.0
82493   -1.0
82494    1.0
82495    1.0
82496    1.0
Name: State, Length: 82497, dtype: float64
splitting dataset into train and test sets...
training started...
[[0.11023622 0.22282609 0.48       ... 0.50275256 0.40259816 1.        ]
 [0.11811024 0.13586957 0.08       ... 0.08785124 0.3761839  1.        ]
 [0.11023622 0.15217391 0.04       ... 0.06245647 0.39304146 1.        ]
 ...
 [0.61417323 0.65217391 0.24       ... 0.0530874  0.0571629  1.        ]
 [0.88976378 0.92391304 0.32       ... 0.04744484 0.00378517 1.        ]
 [0.11811024 0.17934783 0.08       ... 0.1024238  0.38613903 1.        ]]
[-1.  1.  1. ... -1. -1.  1.]
sgd
inf
distances
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
57748
Epoch is: 1 and Cost is: 485.77229234745926
distances
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
57748
Epoch is: 2 and Cost is: 431.5368411840653
distances
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
57748
Epoch is: 4 and 

In [ ]:
from sklearn.svm import SVC
#from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
import pandas

dataframe = pandas.read_table('/content/mixed200.txt')
data = dataframe.drop("No.", axis=1)
data = data.drop("StdPageRank", axis=1)
data = data.drop("VarPageRank", axis=1)
dataset = data.values

# split into input (X) and output (Y) variables
training = 57748
testing = 24749

l = 82497
X = dataset[:,0:]
# print(X.shape)
labels = []
for i in range (0,l):
  if X[i][9] == 1:
    labels.append(1)
  else:
    labels.append(0)

features = X[0:l,0:9]




# Split our data
train, test, train_labels, test_labels = features[0:training],features[training:testing+training],labels[0:training],labels[training:testing+training]
gnb = SVC()
# Train our classifier
print(len(train_labels))
model =gnb.fit(train,train_labels)
preds = gnb.predict(test)
print("----------------------------SVM--------------------------")
print(accuracy_score(test_labels, preds))


57748
----------------------------SVM--------------------------
0.9873126186916643


In [ ]:

import pandas
import numpy as np

dataframe = pandas.read_table('/content/dos200.txt')

print(list(dataframe.min(axis=0)))


[1.0, 10.0, 14.0, 2.0, 1.0, 2.0, 0.0, 0.018149025707461245, 0.046498117468674376, 0.008328770082628284, 0.0]


In [ ]:
#fuzzy

long double Min[10]={-100.0, 10.0, 13.0, 2.0, 0.0, 2.0, 0.0, 0.005301991866120557, 0.02717618228088004, 0.0011771647097430305};
long double Max[10]={-194.0, 132.0, 199.0, 16.0, 1.0, 17.0, 1.0, 0.12536496993782908, 0.1888784481405464, 0.044515632825592434};


#rpm

long double Max[10]={-100.0, 27.0, 86.0, 22.0, 2.0, 20.0, 2.0, 0.12536496993782908, 0.3408619464499469, 0.044515632825592434};
long double Min[10]={-100, 10.0, 13.0, 2.0, 0.0, 2.0, 0.0, 0.02248266777655021, 0.048092654043031015, 0.005555555555555557};

#replay
long double Min[10]={-100.0, 10.0, 13.0, 2.0, 0.0, 2.0, 0.0, 0.015437591081532804, 0.04615917891672077, 0.0034883720930232558};
long double Max[10]={-100, 43.0, 126.0, 13.0, 1.0, 11.0, 1.0, 0.12536496993782908, 0.15252770010406091, 0.044515632825592434};

SyntaxError: ignored

In [ ]:
long double Max[10]={-100.0, 27.0, 100.0, 27.0, 2.0, 27.0, 2.0, 0.12630540975054774, 0.4744080648877346, 0.04184995700600018};
long double Min[10]={-100.0, 10.0, 14.0, 2.0, 1.0, 2.0, 0.0, 0.018149025707461245, 0.046498117468674376, 0.008328770082628284};

SyntaxError: ignored